In [1]:
# Importar librerias
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from pandas.core.arrays import categorical # agregar class pandas.Categorical(values, categories=None, ordered=None, dtype=None, fastpath=False, copy=True)
from sklearn.pipeline import Pipeline #agregar para ejercicio 3a
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
import re

In [2]:
# Importar dataframe 1 "Events downtime"
downtime = pd.read_csv("/content/Ensamble 100D production history DB1 L3W.csv")

In [3]:
# Visualizar df
downtime

,EventID,WorkGroup,AssetName,DownStart,DownEnd,Total Duration,Shift Scheduled Duration,Total Scheduled Duration,Reason,Code,ShiftNumber,ScheduleID,OEE
0,1729797,Ensamble 100D,Bench 100D,5/21/2024,5/21/2024,0:00:02,0:00:02,0:00:02,Falla 12. Scanner de Seguridad Interrumpido Du...,NaN,1,59256,1.0
1,1729790,Ensamble 100D,Bench 100D,5/21/2024,5/21/2024,0:00:29,0:00:29,0:00:29,Falla26. Servo Soporte Derecho Altura Fallo,NaN,1,59256,1.0
2,1729782,Ensamble 100D,Bench 100D,5/21/2024,5/21/2024,0:00:36,0:00:36,0:00:36,Falla35. Servo Pin Izquierdo Longitud Fallo,NaN,1,59256,1.0
3,1729779,Ensamble 100D,Bench 100D,5/21/2024,5/21/2024,0:00:16,0:00:16,0:00:16,Falla 12. Scanner de Seguridad Interrumpido Du...,NaN,1,59256,1.0
4,1729775,Ensamble 100D,Bench 100D,5/21/2024,5/21/2024,0:00:01,0:00:01,0:00:01,Falla35. Servo Pin Izquierdo Longitud Fallo,NaN,1,59256,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,1711475,Ensamble 100D,Bench 100D,5/1/2024,5/1/2024,0:00:16,0:00:16,0:00:16,Falla35. Servo Pin Izquierdo Longitud Fallo,NaN,3,59110,1.0
2080,1711421,Ensamble 100D,Bench 100D,5/1/2024,5/1/2024,0:00:13,0:00:13,0:00:13,Falla35. Servo Pin Izquierdo Longitud Fallo,NaN,3,59110,1.0
2081,1711419,Ensamble 100D,Bench 100D,5/1/2024,5/1/2024,0:00:55,0:00:55,0:00:55,Falla 12. Scanner de Seguridad Interrumpido Du...,NaN,3,59110,1.0
2082,1711417,Ensamble 100D,Bench 100D,5/1/2024,5/1/2024,0:00:05,0:00:05,0:00:05,Falla 12. Scanner de Seguridad Interrumpido Du...,NaN,3,59110,1.0


In [4]:
# contar  valores nulos de dataframe "Events downtime"
downtime.isnull().sum().sort_values(ascending=True)

EventID                        0
WorkGroup                      0
AssetName                      0
DownStart                      0
DownEnd                        0
Total Duration                 0
Shift Scheduled Duration       0
Total Scheduled Duration       0
Reason                         0
ShiftNumber                    0
ScheduleID                     0
OEE                            0
Code                        2084
dtype: int64

In [5]:
# limpiar columnas que no se van a utilizar de dataframe "Events downtime"
downtime.drop(["WorkGroup","Code", "Shift Scheduled Duration", "Total Scheduled Duration"], axis = 1, inplace = True)


In [6]:
downtime

,EventID,AssetName,DownStart,DownEnd,Total Duration,Reason,ShiftNumber,ScheduleID,OEE
0,1729797,Bench 100D,5/21/2024,5/21/2024,0:00:02,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0
1,1729790,Bench 100D,5/21/2024,5/21/2024,0:00:29,Falla26. Servo Soporte Derecho Altura Fallo,1,59256,1.0
2,1729782,Bench 100D,5/21/2024,5/21/2024,0:00:36,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0
3,1729779,Bench 100D,5/21/2024,5/21/2024,0:00:16,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0
4,1729775,Bench 100D,5/21/2024,5/21/2024,0:00:01,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0
...,...,...,...,...,...,...,...,...,...
2079,1711475,Bench 100D,5/1/2024,5/1/2024,0:00:16,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0
2080,1711421,Bench 100D,5/1/2024,5/1/2024,0:00:13,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0
2081,1711419,Bench 100D,5/1/2024,5/1/2024,0:00:55,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0
2082,1711417,Bench 100D,5/1/2024,5/1/2024,0:00:05,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0


In [7]:
# eliminar valores nulos o NAN de tabla downtime

downtime.dropna(inplace=True)
downtime


,EventID,AssetName,DownStart,DownEnd,Total Duration,Reason,ShiftNumber,ScheduleID,OEE
0,1729797,Bench 100D,5/21/2024,5/21/2024,0:00:02,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0
1,1729790,Bench 100D,5/21/2024,5/21/2024,0:00:29,Falla26. Servo Soporte Derecho Altura Fallo,1,59256,1.0
2,1729782,Bench 100D,5/21/2024,5/21/2024,0:00:36,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0
3,1729779,Bench 100D,5/21/2024,5/21/2024,0:00:16,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0
4,1729775,Bench 100D,5/21/2024,5/21/2024,0:00:01,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0
...,...,...,...,...,...,...,...,...,...
2079,1711475,Bench 100D,5/1/2024,5/1/2024,0:00:16,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0
2080,1711421,Bench 100D,5/1/2024,5/1/2024,0:00:13,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0
2081,1711419,Bench 100D,5/1/2024,5/1/2024,0:00:55,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0
2082,1711417,Bench 100D,5/1/2024,5/1/2024,0:00:05,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0


In [8]:
# Renombra la columna Total Duration a Total_Duration

downtime.rename(columns={'Total Duration': 'Total_Duration'}, inplace=True)
downtime


,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE
0,1729797,Bench 100D,5/21/2024,5/21/2024,0:00:02,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0
1,1729790,Bench 100D,5/21/2024,5/21/2024,0:00:29,Falla26. Servo Soporte Derecho Altura Fallo,1,59256,1.0
2,1729782,Bench 100D,5/21/2024,5/21/2024,0:00:36,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0
3,1729779,Bench 100D,5/21/2024,5/21/2024,0:00:16,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0
4,1729775,Bench 100D,5/21/2024,5/21/2024,0:00:01,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0
...,...,...,...,...,...,...,...,...,...
2079,1711475,Bench 100D,5/1/2024,5/1/2024,0:00:16,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0
2080,1711421,Bench 100D,5/1/2024,5/1/2024,0:00:13,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0
2081,1711419,Bench 100D,5/1/2024,5/1/2024,0:00:55,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0
2082,1711417,Bench 100D,5/1/2024,5/1/2024,0:00:05,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0


In [9]:
# convertir columna Total_Duration a segundos, el formato actual es HH:MM:ss, las horas no son horas dia

def convert_to_seconds(time_str):
  hours, minutes, seconds = map(int, time_str.split(':'))
  return hours * 3600 + minutes * 60 + seconds

downtime['Total_Duration'] = downtime['Total_Duration'].apply(convert_to_seconds)


In [10]:
downtime

,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE
0,1729797,Bench 100D,5/21/2024,5/21/2024,2,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0
1,1729790,Bench 100D,5/21/2024,5/21/2024,29,Falla26. Servo Soporte Derecho Altura Fallo,1,59256,1.0
2,1729782,Bench 100D,5/21/2024,5/21/2024,36,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0
3,1729779,Bench 100D,5/21/2024,5/21/2024,16,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0
4,1729775,Bench 100D,5/21/2024,5/21/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0
...,...,...,...,...,...,...,...,...,...
2079,1711475,Bench 100D,5/1/2024,5/1/2024,16,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0
2080,1711421,Bench 100D,5/1/2024,5/1/2024,13,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0
2081,1711419,Bench 100D,5/1/2024,5/1/2024,55,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0
2082,1711417,Bench 100D,5/1/2024,5/1/2024,5,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0


In [11]:
# Separar columna DownStart de dia mes y año
# Separar columna DownStart en  tres columnas mas  una que represente el dia, otra el mes y otra el año

downtime['DownStart_dia'] = pd.to_datetime(downtime['DownStart']).dt.day
downtime['DownStart_mes'] = pd.to_datetime(downtime['DownStart']).dt.month
downtime['DownStart_año'] = pd.to_datetime(downtime['DownStart']).dt.year
downtime


,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE,DownStart_dia,DownStart_mes,DownStart_año
0,1729797,Bench 100D,5/21/2024,5/21/2024,2,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0,21,5,2024
1,1729790,Bench 100D,5/21/2024,5/21/2024,29,Falla26. Servo Soporte Derecho Altura Fallo,1,59256,1.0,21,5,2024
2,1729782,Bench 100D,5/21/2024,5/21/2024,36,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0,21,5,2024
3,1729779,Bench 100D,5/21/2024,5/21/2024,16,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0,21,5,2024
4,1729775,Bench 100D,5/21/2024,5/21/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0,21,5,2024
...,...,...,...,...,...,...,...,...,...,...,...,...
2079,1711475,Bench 100D,5/1/2024,5/1/2024,16,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0,1,5,2024
2080,1711421,Bench 100D,5/1/2024,5/1/2024,13,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0,1,5,2024
2081,1711419,Bench 100D,5/1/2024,5/1/2024,55,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0,1,5,2024
2082,1711417,Bench 100D,5/1/2024,5/1/2024,5,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0,1,5,2024


In [12]:
# Filtrar las filas que contienen 'Falla' o 'Fault' o 'Advertencia' al inicio en la columna 'Reason'
filtered_downtime = downtime[downtime['Reason'].str.contains('^(Falla|Fault|Advertencia)')]

# Reasignar el DataFrame original con el filtrado
downtime = filtered_downtime

# Si deseas resetear los índices del DataFrame
# downtime.reset_index(drop=True, inplace=True)

# Imprimir el DataFrame resultante
print(downtime)


      EventID   AssetName  DownStart    DownEnd  Total_Duration  \
0     1729797  Bench 100D  5/21/2024  5/21/2024               2   
1     1729790  Bench 100D  5/21/2024  5/21/2024              29   
2     1729782  Bench 100D  5/21/2024  5/21/2024              36   
3     1729779  Bench 100D  5/21/2024  5/21/2024              16   
4     1729775  Bench 100D  5/21/2024  5/21/2024               1   
...       ...         ...        ...        ...             ...   
2079  1711475  Bench 100D   5/1/2024   5/1/2024              16   
2080  1711421  Bench 100D   5/1/2024   5/1/2024              13   
2081  1711419  Bench 100D   5/1/2024   5/1/2024              55   
2082  1711417  Bench 100D   5/1/2024   5/1/2024               5   
2083  1711413  Bench 100D   5/1/2024   5/1/2024               1   

                                                 Reason  ShiftNumber  \
0     Falla 12. Scanner de Seguridad Interrumpido Du...            1   
1           Falla26. Servo Soporte Derecho Altura F

<ipython-input-12-e23c3cb4e99f>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_downtime = downtime[downtime['Reason'].str.contains('^(Falla|Fault|Advertencia)')]


In [13]:
# Definir una función para extraer el número de falla
def extract_fault_number(reason):
    # Usar expresión regular para buscar el número de falla
    match = re.search(r'(Falla|Fault|Advertencia)[:.]*\s*(\d+)', reason)
    if match:
        return match.group(2)  # Devolver el segundo grupo que contiene el número de falla
    else:
        return None

# Aplicar la función a la columna "Reason" para extraer el número de falla
downtime["Fault_Number"] = downtime["Reason"].apply(extract_fault_number)

<ipython-input-13-cbb0efe3f7de>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downtime["Fault_Number"] = downtime["Reason"].apply(extract_fault_number)


In [14]:
downtime

,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE,DownStart_dia,DownStart_mes,DownStart_año,Fault_Number
0,1729797,Bench 100D,5/21/2024,5/21/2024,2,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0,21,5,2024,12
1,1729790,Bench 100D,5/21/2024,5/21/2024,29,Falla26. Servo Soporte Derecho Altura Fallo,1,59256,1.0,21,5,2024,26
2,1729782,Bench 100D,5/21/2024,5/21/2024,36,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0,21,5,2024,35
3,1729779,Bench 100D,5/21/2024,5/21/2024,16,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0,21,5,2024,12
4,1729775,Bench 100D,5/21/2024,5/21/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0,21,5,2024,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,1711475,Bench 100D,5/1/2024,5/1/2024,16,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0,1,5,2024,35
2080,1711421,Bench 100D,5/1/2024,5/1/2024,13,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0,1,5,2024,35
2081,1711419,Bench 100D,5/1/2024,5/1/2024,55,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0,1,5,2024,12
2082,1711417,Bench 100D,5/1/2024,5/1/2024,5,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0,1,5,2024,12


In [15]:
# Definir el diccionario de categorías
fault_categories = {
  "Servo Support Faults": ["14", "17", "19", "23", "26", "28", "29", "32", "33", "34", "35", "151"],
  "Servo Pin Faults": ["20"],
  "Pin Position Faults": ["61", "63", "64", "65", "66", "67", "68", "69", "70", "72", "74", "75", "77", "78", "79", "80", "85", "86", "88"],
  "Position Sensor Faults": ["44", "45", "46", "47", "49", "50", "51", "52", "53", "54", "55", "58", "59","90","189"],
  "External Device Faults": ["56","183"],
  "Relay Faults": ["1", "207", "208"],
  "Scanner Faults": ["12", "96"],
  "Electrical Faults": ["2", "97"],
  "Other Faults": ["104","102"]
}

# Función para mapear los valores de la columna "Reason" a las categorías correspondientes
def categorize_reason(reason):
    for category, faults in fault_categories.items():
        if reason in faults:
            return category
    return "Other Faults"  # Si el motivo no coincide con ninguna categoría de otros

# Aplicar la función de categorización a la columna "Reason"
downtime['Category'] = downtime['Fault_Number'].apply(categorize_reason)

# Imprimir el DataFrame con las categorías agregadas
downtime


<ipython-input-15-92aab13925a9>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downtime['Category'] = downtime['Fault_Number'].apply(categorize_reason)


,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE,DownStart_dia,DownStart_mes,DownStart_año,Fault_Number,Category
0,1729797,Bench 100D,5/21/2024,5/21/2024,2,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0,21,5,2024,12,Scanner Faults
1,1729790,Bench 100D,5/21/2024,5/21/2024,29,Falla26. Servo Soporte Derecho Altura Fallo,1,59256,1.0,21,5,2024,26,Servo Support Faults
2,1729782,Bench 100D,5/21/2024,5/21/2024,36,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0,21,5,2024,35,Servo Support Faults
3,1729779,Bench 100D,5/21/2024,5/21/2024,16,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0,21,5,2024,12,Scanner Faults
4,1729775,Bench 100D,5/21/2024,5/21/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0,21,5,2024,35,Servo Support Faults
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,1711475,Bench 100D,5/1/2024,5/1/2024,16,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0,1,5,2024,35,Servo Support Faults
2080,1711421,Bench 100D,5/1/2024,5/1/2024,13,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0,1,5,2024,35,Servo Support Faults
2081,1711419,Bench 100D,5/1/2024,5/1/2024,55,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0,1,5,2024,12,Scanner Faults
2082,1711417,Bench 100D,5/1/2024,5/1/2024,5,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0,1,5,2024,12,Scanner Faults


In [16]:
# Definir el diccionario de categorías
fault_categories = {
  "Servo Support Faults": ["14", "17", "19", "23", "26", "28", "29", "32", "33", "34", "35", "151","146","93","92","91","157"],
  "Servo Pin Faults": ["20","148","197"],
  "Pin Position Faults": ["61", "63", "64", "65", "66", "67", "68", "69", "70", "72", "74", "75", "77", "78", "79", "80", "85", "86", "88","153"],
  "Position Sensor Faults": ["43","44", "45", "46", "47", "49", "50", "51", "52", "53", "54", "55", "58", "59","90","189"],
  "External Device Faults": ["56","183","109","161"],
  "Relay Faults": ["1", "207", "208", "209"],
  "Scanner Faults": ["12", "96"],
  "Electrical Faults": ["2", "97"],
  "Invalid Parameters": ["193","180","178","165"],
  "Other Faults": ["104","102", "193", "98", "93", "91", "125", "101","13"]

}

# Función para mapear los valores de la columna "Reason" a las categorías correspondientes
def categorize_reason(reason):
    for category, faults in fault_categories.items():
        if reason in faults:
            return category
    return "Other Faults"  # Si el motivo no coincide con ninguna categoría de otros

# Aplicar la función de categorización a la columna "Reason"
downtime['Category'] = downtime['Fault_Number'].apply(categorize_reason)

# Imprimir el DataFrame con las categorías agregadas
downtime


<ipython-input-16-c294ba519a15>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downtime['Category'] = downtime['Fault_Number'].apply(categorize_reason)


,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE,DownStart_dia,DownStart_mes,DownStart_año,Fault_Number,Category
0,1729797,Bench 100D,5/21/2024,5/21/2024,2,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0,21,5,2024,12,Scanner Faults
1,1729790,Bench 100D,5/21/2024,5/21/2024,29,Falla26. Servo Soporte Derecho Altura Fallo,1,59256,1.0,21,5,2024,26,Servo Support Faults
2,1729782,Bench 100D,5/21/2024,5/21/2024,36,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0,21,5,2024,35,Servo Support Faults
3,1729779,Bench 100D,5/21/2024,5/21/2024,16,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0,21,5,2024,12,Scanner Faults
4,1729775,Bench 100D,5/21/2024,5/21/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0,21,5,2024,35,Servo Support Faults
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,1711475,Bench 100D,5/1/2024,5/1/2024,16,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0,1,5,2024,35,Servo Support Faults
2080,1711421,Bench 100D,5/1/2024,5/1/2024,13,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0,1,5,2024,35,Servo Support Faults
2081,1711419,Bench 100D,5/1/2024,5/1/2024,55,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0,1,5,2024,12,Scanner Faults
2082,1711417,Bench 100D,5/1/2024,5/1/2024,5,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0,1,5,2024,12,Scanner Faults


In [17]:
# Contar la frecuencia de cada valor en la columna 'DayOfWeek'
frequency_counts = downtime['Category'].value_counts()

# Mostrar la frecuencia de cada valor
print("Frecuencia de cada valor en la columna 'Category':")
print(frequency_counts)

Frecuencia de cada valor en la columna 'Category':
Category
Servo Support Faults      1092
Scanner Faults             683
Position Sensor Faults      95
Pin Position Faults         77
Relay Faults                31
Other Faults                 8
Electrical Faults            3
Servo Pin Faults             2
Name: count, dtype: int64


In [18]:
# Agrega una columna en el DF de downtime donde evalue la columna Category y le asigne un numero para cada categoria, esta nueva columna se llamara "Category_Number" y sera del tipo int y comenzara a partir del 0

# Crear un diccionario que mapee las categorías a números
category_to_number = {
    "Servo Support Faults": 0,
    "Servo Pin Faults": 1,
    "Pin Position Faults": 2,
    "Position Sensor Faults": 3,
    "External Device Faults": 4,
    "Relay Faults": 5,
    "Scanner Faults": 6,
    "Electrical Faults": 7,
    "Invalid Parameters": 8,
    "Other Faults": 9
}

# Agregar una nueva columna "Category_Number" al DataFrame
downtime["Category_Number"] = downtime["Category"].map(category_to_number).astype(int)

# Imprimir el DataFrame con la nueva columna
print(downtime)


      EventID   AssetName  DownStart    DownEnd  Total_Duration  \
0     1729797  Bench 100D  5/21/2024  5/21/2024               2   
1     1729790  Bench 100D  5/21/2024  5/21/2024              29   
2     1729782  Bench 100D  5/21/2024  5/21/2024              36   
3     1729779  Bench 100D  5/21/2024  5/21/2024              16   
4     1729775  Bench 100D  5/21/2024  5/21/2024               1   
...       ...         ...        ...        ...             ...   
2079  1711475  Bench 100D   5/1/2024   5/1/2024              16   
2080  1711421  Bench 100D   5/1/2024   5/1/2024              13   
2081  1711419  Bench 100D   5/1/2024   5/1/2024              55   
2082  1711417  Bench 100D   5/1/2024   5/1/2024               5   
2083  1711413  Bench 100D   5/1/2024   5/1/2024               1   

                                                 Reason  ShiftNumber  \
0     Falla 12. Scanner de Seguridad Interrumpido Du...            1   
1           Falla26. Servo Soporte Derecho Altura F

<ipython-input-18-e45b3244ebc1>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downtime["Category_Number"] = downtime["Category"].map(category_to_number).astype(int)


In [19]:
#imprime el DF con columna nueva
downtime

,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE,DownStart_dia,DownStart_mes,DownStart_año,Fault_Number,Category,Category_Number
0,1729797,Bench 100D,5/21/2024,5/21/2024,2,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0,21,5,2024,12,Scanner Faults,6
1,1729790,Bench 100D,5/21/2024,5/21/2024,29,Falla26. Servo Soporte Derecho Altura Fallo,1,59256,1.0,21,5,2024,26,Servo Support Faults,0
2,1729782,Bench 100D,5/21/2024,5/21/2024,36,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0,21,5,2024,35,Servo Support Faults,0
3,1729779,Bench 100D,5/21/2024,5/21/2024,16,Falla 12. Scanner de Seguridad Interrumpido Du...,1,59256,1.0,21,5,2024,12,Scanner Faults,6
4,1729775,Bench 100D,5/21/2024,5/21/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,1,59256,1.0,21,5,2024,35,Servo Support Faults,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,1711475,Bench 100D,5/1/2024,5/1/2024,16,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0,1,5,2024,35,Servo Support Faults,0
2080,1711421,Bench 100D,5/1/2024,5/1/2024,13,Falla35. Servo Pin Izquierdo Longitud Fallo,3,59110,1.0,1,5,2024,35,Servo Support Faults,0
2081,1711419,Bench 100D,5/1/2024,5/1/2024,55,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0,1,5,2024,12,Scanner Faults,6
2082,1711417,Bench 100D,5/1/2024,5/1/2024,5,Falla 12. Scanner de Seguridad Interrumpido Du...,3,59110,1.0,1,5,2024,12,Scanner Faults,6


In [20]:
# prompt: Imprime el tipo de dato para cada columna

for column in downtime.columns:
    print(f"{column}: {downtime[column].dtype}")


EventID: int64
AssetName: object
DownStart: object
DownEnd: object
Total_Duration: int64
Reason: object
ShiftNumber: int64
ScheduleID: int64
OEE: float64
DownStart_dia: int32
DownStart_mes: int32
DownStart_año: int32
Fault_Number: object
Category: object
Category_Number: int64


In [21]:
# Transformar las columnas DownStart y DownEnd a tipo Fechas, la columna Fault_Number a int, muestra el data frame downtime

import pandas as pd
# Transformar las columnas DownStart y DownEnd a tipo Fechas
downtime['DownStart'] = pd.to_datetime(downtime['DownStart'])
downtime['DownEnd'] = pd.to_datetime(downtime['DownEnd'])

# Transformar la columna Fault_Number a int
downtime['Fault_Number'] = downtime['Fault_Number'].astype(int)

# Mostrar el DataFrame downtime
print(downtime)


<ipython-input-21-1a17fc92f5f2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downtime['DownStart'] = pd.to_datetime(downtime['DownStart'])
<ipython-input-21-1a17fc92f5f2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downtime['DownEnd'] = pd.to_datetime(downtime['DownEnd'])
<ipython-input-21-1a17fc92f5f2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

      EventID   AssetName  DownStart    DownEnd  Total_Duration  \
0     1729797  Bench 100D 2024-05-21 2024-05-21               2   
1     1729790  Bench 100D 2024-05-21 2024-05-21              29   
2     1729782  Bench 100D 2024-05-21 2024-05-21              36   
3     1729779  Bench 100D 2024-05-21 2024-05-21              16   
4     1729775  Bench 100D 2024-05-21 2024-05-21               1   
...       ...         ...        ...        ...             ...   
2079  1711475  Bench 100D 2024-05-01 2024-05-01              16   
2080  1711421  Bench 100D 2024-05-01 2024-05-01              13   
2081  1711419  Bench 100D 2024-05-01 2024-05-01              55   
2082  1711417  Bench 100D 2024-05-01 2024-05-01               5   
2083  1711413  Bench 100D 2024-05-01 2024-05-01               1   

                                                 Reason  ShiftNumber  \
0     Falla 12. Scanner de Seguridad Interrumpido Du...            1   
1           Falla26. Servo Soporte Derecho Altura F

In [22]:
# Verifica valores Nulos en las columnas y NAN

downtime.isnull().sum()


EventID            0
AssetName          0
DownStart          0
DownEnd            0
Total_Duration     0
Reason             0
ShiftNumber        0
ScheduleID         0
OEE                0
DownStart_dia      0
DownStart_mes      0
DownStart_año      0
Fault_Number       0
Category           0
Category_Number    0
dtype: int64

In [23]:
# Agrupa las columnas de la siguiente manera : EventID, ScheduleID, AssetName, DownStart, DownEnd, Reason, Category, DownStart_dia, DownStart_mes, DownStart_año, Total_Duration, Fault_Number, Category_Number, OEE

downtime = downtime.groupby(['EventID', 'ScheduleID', 'AssetName', 'DownStart', 'DownEnd', 'Reason', 'Category', 'DownStart_dia', 'DownStart_mes', 'DownStart_año', 'Total_Duration', 'Fault_Number', 'Category_Number','ShiftNumber']).agg({'OEE': 'mean'})

print(downtime)


                                                                                                                                                                                                                               OEE
EventID ScheduleID AssetName  DownStart  DownEnd    Reason                                             Category             DownStart_dia DownStart_mes DownStart_año Total_Duration Fault_Number Category_Number ShiftNumber     
1711413 59110      Bench 100D 2024-05-01 2024-05-01 Falla35. Servo Pin Izquierdo Longitud Fallo        Servo Support Faults 1             5             2024          1              35           0               3            1.0
1711417 59110      Bench 100D 2024-05-01 2024-05-01 Falla 12. Scanner de Seguridad Interrumpido Dur... Scanner Faults       1             5             2024          5              12           6               3            1.0
1711419 59110      Bench 100D 2024-05-01 2024-05-01 Falla 12. Scanner de Seguridad Interrump

In [24]:
# Guarda el data set depurado
downtime.to_csv("/content/downtime.csv", index= True, header = True)
